In [12]:
import cv2
import easyocr
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

In [2]:
img = cv2.imread("/home/dracula/Documents/pdf_to_data_frame_internship/Pinnacle_Assignment_Pdf_Folder_Pdf_1/33.jpg")

As per the task,the challenge was to extract the data from the pdf and put it in the dataframe, while doing the task my approach was as follows, 
* conver the pdf into jpg img
* Then get the biggest contour from the image and crop it and save it, this will be genrally ge the table which will have the biggest contour,
* Then use tabula library to detect the dataframe from the image and convert it into a dataframe.

but the challenge i faced is that tabula cant work in scanned image which later turned into a pdf, even i tried with camelot it didnt worked, 
here is the sample code,

In [3]:
#this code will crop the biggest contour and then crop it,generally the tabuls
image= cv2.imread('/home/dracula/Documents/pdf_to_data_frame_internship/Pinnacle_Assignment_Pdf_Folder_Pdf_3/omg.jpg')
original_image= image

gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

edges= cv2.Canny(gray, 50, 200)

contours, hierarchy= cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

def get_contour_areas(contours):
    all_areas= []
    for cnt in contours:
        area= cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

sorted_contours= sorted(contours, key=cv2.contourArea, reverse=True)

# Get the largest item
largest_item= sorted_contours[0]

# Find the largest contour and its bounding rectangle
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)

# Crop image to ROI and extract text using EasyOCR
roi = image[y:y+h, x:x+w]
reader = easyocr.Reader(['en'])
result = reader.readtext(roi)

# Print the extracted text
print(result)

# Save cropped image as JPEG
cv2.imwrite('cropped.jpg', roi)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[10, 16], [62, 16], [62, 40], [10, 40]], 'Item', 0.999304473400116), ([[75, 13], [194, 13], [194, 45], [75, 45]], 'Description', 0.9997169947660091), ([[497, 9], [590, 9], [590, 40], [497, 40]], 'Quantity', 0.999982356947327), ([[714, 8], [816, 8], [816, 34], [714, 34]], 'Unit Price', 0.8741189794944534), ([[863, 9], [975, 9], [975, 29], [863, 29]], 'Total Price', 0.7860219404069949), ([[76, 42], [218, 42], [218, 66], [76, 66]], 'Product Code', 0.8025055489767547), ([[410, 42], [492, 42], [492, 66], [410, 66]], 'Ordered', 0.9986553459982818), ([[516, 42], [594, 42], [594, 68], [516, 68]], 'Shipped', 0.9999419211418838), ([[624, 40], [664, 40], [664, 64], [624, 64]], 'B/O', 0.49711920779781776), ([[742, 36], [784, 36], [784, 60], [742, 60]], 'RM', 0.9999148644275015), ([[898, 34], [940, 34], [940, 58], [898, 58]], 'RM', 0.9998490333265065), ([[76, 80], [214, 80], [214, 108], [76, 108]], 'Supplier Item:', 0.9957018890946929), ([[721, 71], [806, 71], [806, 99], [721, 99]], '4,219.50', 

True

Different approach! 


Here i have used Easyocr for text detection, and getting the bounding box coordinate of the text detected, and then
The first loop creates a list called table_data that will contain only the extracted text (not the bounding box coordinates). It iterates through each element in result (which contains both the text and coordinates) and appends the text to table_data. The resulting list of lists is then used to create a new DataFrame called df with a single column named "Header".

The second loop iterates through each coordinate in text_coords (which contains the bounding box coordinates for each text element) and attempts to match each coordinate with a cell in the DataFrame df. It does this by iterating through each element in result and checking the bounding box coordinates against the current coordinate in text_coords.

If the y-coordinate of the bounding box matches the current coordinate, the text is assigned to the cell in the second column of the DataFrame (df.iloc[i, 1]). If the x-coordinate matches the current coordinate, the text is assigned to the cell in the first column of the DataFrame (df.iloc[i, 0]). If the x-coordinate falls between the first and last coordinates of the current coordinate, the text is assigned to the appropriate column in the DataFrame (df.iloc[i, df.columns.get_loc('Column {}'.format(i+1))]). The resulting DataFrame should contain the extracted text in the correct order and position based on the layout of the image.


In [4]:
# Use EasyOCR to extract the text from the image and get the text coordinates
reader = easyocr.Reader(['en'])
result = reader.readtext('/home/dracula/Documents/pdf_to_data_frame_internship/cropped.jpg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [5]:
text_coords = []
for corr,text,pro in result:
    text_coords.append(corr)

In [6]:
# Create a new image with the text positioned correctly, after detecting the largest contour
# and then cropping the image.
new_img = Image.fromarray(img)
draw = ImageDraw.Draw(new_img)
for coord in enumerate(text_coords):
    coor_d=coord[1][0]
    i=coord[0]
    coo_x=int(coor_d[0])
    coo_y=int(coor_d[1])
    draw.text((coo_x,coo_y),result[i], fill=(0, 0, 0))
# # # Save the new image
new_img.save('new_table.jpg')

In [9]:
# Create a pandas dataframe from the extracted data
table_data = []
for text in result:
    table_data.append([text[1]])  # add only the text, not the bounding box coordinates
df = pd.DataFrame(table_data, columns=['Header'])

# Add columns to the dataframe based on the position of the text
for i, coord in enumerate(text_coords):
    if i > len(text_coords) - len(df):
        break
    df.insert(loc=i+1, column='Column {}'.format(i+1), value='')
    for text in result:
        text_box = text[0]
        if text_box[1] == coord[1]:
            df.iloc[i, 1] = text[1]
        elif text_box[0] == coord[0]:
            df.iloc[i, 0] = text[1]
        elif coord[0] < text_box[0] < coord[2]:
            df.iloc[i, df.columns.get_loc('Column {}'.format(i+1))] = text[1]

later we can, manupulate this df, for further analysis.

In [11]:
df

,Header,Column 1
0,Description,0% SST
1,Quantity,
2,Unit Price,
3,Total Price,
4,Product Code,
5,Ordered,
6,Shipped,
7,B/O,
8,RM,
9,RM,
